# Tutorial 04: Running an anomaly detector

In tutorials 1 and 2 we've seen how to initialize a simulation and run a single instance, and how to design an adversarial agent to degrade the traffic flow. In this tutorial we will look at setting up running a set of simulations, which allows us to vary different paremeters of interest.